### RAG Casestudy (CSV)

In [5]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.vectorstores import Qdrant
from langchain.indexes import VectorstoreIndexCreator
from langchain.chains import RetrievalQA
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate

In [6]:
loader = CSVLoader(file_path='./data/movies.csv',
                   source_column='original_title',
                   encoding='utf-8',
                   csv_args={'delimiter': ',', 'fieldnames': ['id', 'original_language', 'original_title', 'popularity', 'release_date', 'vote_average', 'vote_count', 'genre', 'overview', 'revenue', 'runtime', 'tagline']})

data = loader.load()

In [7]:
print('Loaded %s movies' % len(data))

Loaded 501 movies


In [8]:
openai_api_key = os.environ["OPENAI_API_KEY"]
model_name = "gpt-3.5-turbo-0125"
embedding_model_name = "text-embedding-3-large"

In [9]:
embeddings = OpenAIEmbeddings(
    model=embedding_model_name,
    openai_api_key=openai_api_key
)

In [10]:
llm = ChatOpenAI(
    model=model_name,
    openai_api_key=openai_api_key,
    temperature=0.1,
    max_tokens=1000
)

In [11]:
url = "http://localhost:6333"

In [13]:
qdrant = Qdrant.from_documents(
    data,
    embeddings,
    url=url,
    prefer_grpc=False,
    collection_name="my_movies",
)

In [14]:
vectorstore = qdrant

In [15]:
query = "Can you suggest similar movies to The Matrix?"

In [16]:
query_results = qdrant.similarity_search(query)

In [17]:
for doc in query_results:
    print(doc.metadata['source'])

Source Code
Source Code
Paycheck
Paycheck


In [18]:
index_creator = VectorstoreIndexCreator(embedding=embeddings, vectorstore_cls=Qdrant)
docsearch = index_creator.from_loaders([loader])

In [19]:
chain = RetrievalQA.from_chain_type(llm=llm,
                                    chain_type="stuff",
                                    retriever=docsearch.vectorstore.as_retriever(),
                                    input_key="question",
                                    return_source_documents=True)

In [20]:
query = "Do you have a column called popularity?"
response = chain.invoke({"question": query})

In [21]:
print(response['result'])

Yes, there is a column called "popularity" in the dataset.


In [22]:
print(response['source_documents'])

[Document(metadata={'source': 'original_title', 'row': 0, '_id': '084eb372-726a-4336-902b-19d1d2265519', '_collection_name': '7b98ac3fae9549669e01bea1565abe85'}, page_content='id: id\noriginal_language: original_language\noriginal_title: original_title\npopularity: popularity\nrelease_date: release_date\nvote_average: vote_average\nvote_count: vote_count\ngenre: genre\noverview: overview\nrevenue: revenue\nruntime: runtime\ntagline: tagline'), Document(metadata={'source': 'Fame', 'row': 321, '_id': 'c7c0a055-e0d4-4d8a-99d7-71b2ef393680', '_collection_name': '7b98ac3fae9549669e01bea1565abe85'}, page_content="id: 3537.0\noriginal_language: en\noriginal_title: Fame\npopularity: 7.651\nrelease_date: 1980-05-16\nvote_average: 6.5\nvote_count: 320.0\ngenre: ['Drama', 'Music']\noverview: A chronicle of the lives of several teenagers who attend a New York high school for students gifted in the performing arts.\nrevenue: 42000000.0\nruntime: 134.0\ntagline: If they've really got what it takes, 

In [24]:
query = """If the popularity score is defined as a higher value being a more popular movie,
what is the name of the most popular movie in the data provided?"""
response = chain.invoke({"question": query})

In [25]:
print(response['result'])

The most popular movie in the data provided is "Ben-Hur" with a popularity score of 35.076.


In [26]:
print(response['source_documents'])

[Document(metadata={'source': 'Casablanca', 'row': 483, '_id': 'a022beef-e6bf-4368-88ca-2e115dd511cf', '_collection_name': '7b98ac3fae9549669e01bea1565abe85'}, page_content="id: 289.0\noriginal_language: en\noriginal_title: Casablanca\npopularity: 22.587\nrelease_date: 1942-11-26\nvote_average: 8.2\nvote_count: 3961.0\ngenre: ['Drama', 'Romance']\noverview: In Casablanca, Morocco in December 1941, a cynical American expatriate meets a former lover, with unforeseen complications.\nrevenue: 10462500.0\nruntime: 102.0\ntagline: They had a date with fate in Casablanca!"), Document(metadata={'source': 'Ben-Hur', 'row': 226, '_id': '285e3ebf-63c2-4680-b07e-60574094e229', '_collection_name': '7b98ac3fae9549669e01bea1565abe85'}, page_content="id: 665.0\noriginal_language: en\noriginal_title: Ben-Hur\npopularity: 35.076\nrelease_date: 1959-11-18\nvote_average: 7.8\nvote_count: 1938.0\ngenre: ['Action', 'Drama', 'History']\noverview: In 25 AD,Judah Ben-Hur, a Jew in ancient Judea, opposes the oc

In [27]:
query = """find the spider man movie released in 2002?"""
response = chain.invoke({"question": query})
print(response['result'])
print(response['source_documents'])

The Spider-Man movie released in 2002 is simply titled "Spider-Man."
[Document(metadata={'source': 'Spider-Man: Far From Home', 'row': 353, '_id': 'bdae1fde-65a8-429c-bbbf-c75e7d88e83d', '_collection_name': '7b98ac3fae9549669e01bea1565abe85'}, page_content="id: 429617.0\noriginal_language: en\noriginal_title: Spider-Man: Far From Home\npopularity: 273.535\nrelease_date: 2019-06-28\nvote_average: 7.5\nvote_count: 10893.0\ngenre: ['Action', 'Adventure', 'Science Fiction']\noverview: Peter Parker and his friends go on a summer trip to Europe. However, they will hardly be able to rest - Peter will have to agree to help Nick Fury uncover the mystery of creatures that cause natural disasters and destruction throughout the continent.\nrevenue: 1131927996.0\nruntime: 129.0\ntagline: It’s time to step up."), Document(metadata={'source': 'Men in Black II', 'row': 358, '_id': '45eae049-c1b3-49d4-8868-0588c215af01', '_collection_name': '7b98ac3fae9549669e01bea1565abe85'}, page_content="id: 608.0\n

In [28]:
query = """do you know Bangalore?"""
response = chain.invoke({"question": query})
print(response['result'])
print(response['source_documents'])

Yes, Bangalore is a city in southern India and is the capital of the state of Karnataka. It is known for its pleasant climate, vibrant nightlife, and being a hub for technology companies and startups.
[Document(metadata={'source': 'कुछ कुछ होता है', 'row': 409, '_id': 'a43b755f-8aad-4dd9-8c7e-1a036c3fd956', '_collection_name': '7b98ac3fae9549669e01bea1565abe85'}, page_content="id: 11854.0\noriginal_language: hi\noriginal_title: कुछ कुछ होता है\npopularity: 9.783\nrelease_date: 1998-10-16\nvote_average: 7.7\nvote_count: 290.0\ngenre: ['Drama', 'Romance']\noverview: Anjali is left heartbroken when her best friend and secret crush, Rahul, falls in love with Tina. Years later, Tina's young daughter tries to fulfil her mother's last wish of uniting Rahul and Anjali.\nrevenue: 15306000.0\nruntime: 185.0\ntagline: Love is Friendship."), Document(metadata={'source': 'Blood Money', 'row': 79, '_id': 'e4363a9a-76c7-4bc9-9d57-0a5e1ae9206b', '_collection_name': '7b98ac3fae9549669e01bea1565abe85'},

In [29]:
prompt_template="""
Use the following piece of context to answer the question asked.
Please try to provide the answer only based on the context.
Do not use any other sources of information.
If you do not know the answer, please respond to say that I do not know. 

{context}
Question:{question}

Helpful Answers:
 """

In [30]:
prompt=PromptTemplate(template=prompt_template,input_variables=["context","question"])

In [31]:
chain = RetrievalQA.from_chain_type(llm=llm,
                                    chain_type="stuff",
                                    retriever=docsearch.vectorstore.as_retriever(),
                                    input_key="question",
                                    return_source_documents=True,
                                    chain_type_kwargs={"prompt": prompt
                                                       })

In [32]:
query = "Do you have a column called popularity?"
response = chain.invoke({"question": query})

print(response['result'])

Yes, the column called popularity is present in the context provided.


In [ ]:
query = """do you know Bangalore?"""
response = chain.invoke({"question": query})
print(response['result'])
print(response['source_documents'])


I do not know.
[Document(metadata={'source': 'कुछ कुछ होता है', 'row': 409, '_id': 'a43b755f-8aad-4dd9-8c7e-1a036c3fd956', '_collection_name': '7b98ac3fae9549669e01bea1565abe85'}, page_content="id: 11854.0\noriginal_language: hi\noriginal_title: कुछ कुछ होता है\npopularity: 9.783\nrelease_date: 1998-10-16\nvote_average: 7.7\nvote_count: 290.0\ngenre: ['Drama', 'Romance']\noverview: Anjali is left heartbroken when her best friend and secret crush, Rahul, falls in love with Tina. Years later, Tina's young daughter tries to fulfil her mother's last wish of uniting Rahul and Anjali.\nrevenue: 15306000.0\nruntime: 185.0\ntagline: Love is Friendship."), Document(metadata={'source': 'Blood Money', 'row': 79, '_id': 'e4363a9a-76c7-4bc9-9d57-0a5e1ae9206b', '_collection_name': '7b98ac3fae9549669e01bea1565abe85'}, page_content="id: 105379.0\noriginal_language: hi\noriginal_title: Blood Money\npopularity: 38.939\nrelease_date: 2012-03-31\nvote_average: 6.1\nvote_count: 7.0\ngenre: ['Drama', 'Actio

In [34]:
query = """find the spider man movie released in 2002"""
response = chain.invoke({"question": query})
print(response['result'])
print(response['source_documents'])

I do not know.
[Document(metadata={'source': 'Spider-Man: Far From Home', 'row': 353, '_id': 'bdae1fde-65a8-429c-bbbf-c75e7d88e83d', '_collection_name': '7b98ac3fae9549669e01bea1565abe85'}, page_content="id: 429617.0\noriginal_language: en\noriginal_title: Spider-Man: Far From Home\npopularity: 273.535\nrelease_date: 2019-06-28\nvote_average: 7.5\nvote_count: 10893.0\ngenre: ['Action', 'Adventure', 'Science Fiction']\noverview: Peter Parker and his friends go on a summer trip to Europe. However, they will hardly be able to rest - Peter will have to agree to help Nick Fury uncover the mystery of creatures that cause natural disasters and destruction throughout the continent.\nrevenue: 1131927996.0\nruntime: 129.0\ntagline: It’s time to step up."), Document(metadata={'source': 'Men in Black II', 'row': 358, '_id': '45eae049-c1b3-49d4-8868-0588c215af01', '_collection_name': '7b98ac3fae9549669e01bea1565abe85'}, page_content="id: 608.0\noriginal_language: en\noriginal_title: Men in Black II